# Program the idm_stages ontology

The developments of the buildingSMART Norway Guide inlcuded the adoption of stages, which was based on the IDM standard (ISO 29481-1:2010 Building information modelling). This is a program to make a machine readable ontology of these, and its mappings. We'd name it idm_stages. 

In this notebook we will be programming the bSN Stages (prosjektfaseinndeling) and its mappings to Norsk Fasenorm and RIBA Plan of Work, as found in [definitions folder](definitions).

We will also provide relevant mappings as link sets and will be using the [EUROTL Framework](https://www.roadotl.eu/static/eurotl-ontologies/index.html) by subclassing [Life Cycle Stage](https://eurotl.laces-viewer.tech/#/view?uri=http%3A%2F%2Fwww.roadotl.eu%2Fdef%2FLifecyclePhase)

In [1]:
# We use xlrd library to parse the excel file containing the stages definitions and mappings to other stage/phase models
import xlrd
# We use os for file and folder location functionality in the script. 
import os

Ensure that you use the correct path to access the file on your system!

In [2]:
currentPath = os.getcwd()
pathFolderData = os.path.abspath( os.path.join( currentPath , os.pardir ) )
pathToFile = os.path.join( pathFolderData , "definitions" , "bsng_faser_fn_pow.xlsx" )
print( "You specify the path to the file as: " , pathToFile )

You specify the path to the file as:  /Users/Sigve/Projects/bSN/lifecyclestagesontology/definitions/bsng_faser_fn_pow.xlsx


In [3]:
wb = xlrd.open_workbook( pathToFile )

print( "This are the sheets in you file: " )
print( wb.sheet_names() )

This are the sheets in you file: 
['Stages']


In [4]:
stages_sheet = wb.sheet_by_name( "Stages" )
print(stages_sheet.cell_value(0,0)+"\n")
print("Number of columns: {}\nNumber of Rows: {}".format(stages_sheet.ncols,stages_sheet.nrows))

buildingSMART Norge Guiden
Prosjektfaseinndeling
Basert på ISO 29481-1
Koblet med Norsk fasenorm og RIBA Plan of Work faseinndeling

Number of columns: 12
Number of Rows: 17


In [5]:
# Headings for respective bSN, Fasenorm and Riba Plan of works. 
for i in range(stages_sheet.ncols):
    print("Column nr: "+str(i)+"\t"+stages_sheet.cell_value(2,i))

Column nr: 0	Fase/Stage ID
Column nr: 1	Fase navn norsk
Column nr: 2	Fase beskrivelse
Column nr: 3	Stage name English
Column nr: 4	Stage description
Column nr: 5	
Column nr: 6	Fasenorm steg
Column nr: 7	Fasenorm faser
Column nr: 8	
Column nr: 9	PoW stages
Column nr: 10	Plan of Work Core objective
Column nr: 11	PoW stage color code


In [6]:
# Mappings between IDM , Fase Norm and Plan of Work
for i in range(3,stages_sheet.nrows):
    idm_stage = stages_sheet.cell_value(i,0)
    next_stage = stages_sheet.cell_value(i,6)
    riba_stage = stages_sheet.cell_value(i,9)
    if len(idm_stage) < 5: ## Print formatting
        idm_stage = idm_stage + "  "
    j = i   
    while next_stage == "":
        j = j-1
        next_stage = stages_sheet.cell_value(j,6)
    k = i
    while riba_stage == "":
        k = k-1
        riba_stage = stages_sheet.cell_value(k,9)
    string = "IDM Stage: {}:  Neste Steg {}:    RIBA Plan Of Wok {}".format(idm_stage,next_stage,riba_stage)
    border = len(string)*"-"
    print(string)
    print(border)

IDM Stage: S00  :  Neste Steg 1.0:    RIBA Plan Of Wok 0.0
----------------------------------------------------------
IDM Stage: S01  :  Neste Steg 1.0:    RIBA Plan Of Wok 0.0
----------------------------------------------------------
IDM Stage: S02  :  Neste Steg 2.0:    RIBA Plan Of Wok 1.0
----------------------------------------------------------
IDM Stage: S03  :  Neste Steg 2.0:    RIBA Plan Of Wok 1.0
----------------------------------------------------------
IDM Stage: S04  :  Neste Steg 3.0:    RIBA Plan Of Wok 2.0
----------------------------------------------------------
IDM Stage: S05  :  Neste Steg 3.0:    RIBA Plan Of Wok 3.0
----------------------------------------------------------
IDM Stage: S06.1:  Neste Steg 4.0:    RIBA Plan Of Wok 4.0
----------------------------------------------------------
IDM Stage: S06.2:  Neste Steg 4.0:    RIBA Plan Of Wok 4.0
----------------------------------------------------------
IDM Stage: S07  :  Neste Steg 4.0:    RIBA Plan Of Wok 4

## Creating the IDM_Stages onotlogy

In [7]:
# Import needed components from rdflib
from rdflib import Graph , Literal , BNode , Namespace , RDF , RDFS , OWL , URIRef

# initiate triple store, i.e. Graph()
g = Graph()

# Add namespaces
g.bind( "owl" , OWL )
EUROTL = Namespace( "http://www.roadotl.eu/def/" )
g.bind( "eurotl" , EUROTL )
NS = Namespace( "https://ont.buildingsmart.no/idm_stages/" )
g.bind( "idm_stage" , NS )

# Initiate ontology entity
s = URIRef( "https://ont.buildingsmart.no/idm_stages/" )
p = RDF.type
o = OWL.ontology
g.add( ( s , p , o ) )

In [8]:
for i in range(3,stages_sheet.nrows):
    
    stageID = str( stages_sheet.cell_value( i , 0 ) ).replace(".","_")
    #print( "Stage ID:" , stageID )
    s = NS[ "Stage_"+stageID ]
    p = RDF.type
    o = EUROTL[ "LifecyclePhase" ]
    g.add( ( s , p , o ) )
    
    stage_name = str( stages_sheet.cell_value( i , 1 ))
    #print( "Navn:" , stage_name )
    p = RDFS.label
    o = Literal(stage_name, lang="no" )
    g.add( ( s , p , o ) )
    
    stage_description_no = str( stages_sheet.cell_value( i , 2 ))
    #print( "Beskrivelse:" , stage_description_no )
    p = RDFS.comment
    o = Literal(stage_description_no, lang="no" )
    g.add( ( s , p , o ) )
    
    stage_name_en = str( stages_sheet.cell_value( i , 3 ))
    #print( "Name:" , stage_name_en )
    p = RDFS.label
    o = Literal(stage_name_en, lang="en" )
    g.add( ( s , p , o ) )
    
    stage_description_en = str( stages_sheet.cell_value( i , 4 ))
    #print( "Description:" , stage_description_en )
    p = RDFS.comment
    o = Literal(stage_description_en, lang="en" )
    g.add( ( s , p , o ) )
    
    

In [9]:
# print file
g.serialize( destination = "../filer/idm_stages.ttl" , format = "turtle" )
print( "Created idm_stages.ttl" )

Created idm_stages.ttl


# Neste Steg Ontologi

In [10]:
# initiate triple store, i.e. Graph()
g2 = Graph()

# Add namespaces
g2.bind( "owl" , OWL )
EUROTL = Namespace( "http://www.roadotl.eu/def/" )
g2.bind( "eurotl" , EUROTL )
NS = Namespace( "https://ont.buildingsmart.no/fasenorm/" )
g2.bind( "fasenorm" , NS )

# Initiate ontology entity
s = URIRef( "https://ont.buildingsmart.no/fasenorm/" )
p = RDF.type
o = OWL.ontology
g2.add( ( s , p , o ) )

In [11]:
for i in range(3,stages_sheet.nrows):
    
    if stages_sheet.cell_value( i , 6 ):
        stageID = str( stages_sheet.cell_value( i , 6 ) ).split(".")[0]
        #print( "Stage ID:" , "Steg"+stageID )
        s = NS[ "Steg_"+stageID ]
        p = RDF.type
        o = EUROTL[ "LifecyclePhase" ]
        g2.add( ( s , p , o ) )

        stage_name = str( stages_sheet.cell_value( i , 7 ))
        #print( "Navn:" , stage_name )
        p = RDFS.label
        o = Literal(stage_name, lang="no" )
        g2.add( ( s , p , o ) )

In [12]:
# print file
g2.serialize( destination = "../filer/neste_steg.ttl" , format = "turtle" )
print( "Created neste_steg.ttl" )

Created neste_steg.ttl


# Plan of work Ontology

In [13]:
# initiate triple store, i.e. Graph()
g3 = Graph()

# Add namespaces
g3.bind( "owl" , OWL )
EUROTL = Namespace( "http://www.roadotl.eu/def/" )
g3.bind( "eurotl" , EUROTL )
NS = Namespace( "https://ont.buildingsmart.no/plan_of_work/" )
g3.bind( "plan_of_work" , NS )

# Initiate ontology entity
s = URIRef( "https://ont.buildingsmart.no/plan_of_work/" )
p = RDF.type
o = OWL.ontology
g3.add( ( s , p , o ) )

In [14]:
for i in range(3,stages_sheet.nrows):
    stageID = str( stages_sheet.cell_value( i , 9 ) ).split(".")[0]
    stage_name = str( stages_sheet.cell_value( i , 10 ))
    
    if stageID not in ("","N/A"):
        #print( "Stage ID:" , "Stage_"+stageID )
        s = NS[ "Stage_"+stageID ]
        p = RDF.type
        o = EUROTL[ "LifecyclePhase" ]
        g3.add( ( s , p , o ) )
        
        #print( "Name:" , stage_name )
        p = RDFS.label
        o = Literal(stage_name, lang="en" )
        g3.add( ( s , p , o ) )

In [15]:
# print file
g3.serialize( destination = "../filer/plan_of_work.ttl" , format = "turtle" )
print( "Created plan_of_work.ttl" )

Created plan_of_work.ttl


# Linkset IDM_stage - Fasenorm and IDM_Stage - Plan of Work

In [16]:
# initiate triple store, i.e. Graph()
g4 = Graph()

# Add namespaces
g4.bind( "owl" , OWL )
idm_stages = Namespace( "https://ont.buildingsmart.no/idm_stages/" )
g4.bind( "idm_stage" , idm_stages )
fasenorm = Namespace( "https://ont.buildingsmart.no/fasenorm/" )
g4.bind( "fasenorm" , fasenorm )
plan_of_work = Namespace( "https://ont.buildingsmart.no/plan_of_work/" )
g4.bind( "plan_of_work" , plan_of_work )

NS = Namespace( "https://ont.buildingsmart.no/idm_stages_alignment/" )
g4.bind( "idm_stages_alignment" , NS )

# Initiate ontology entity
s = URIRef( "https://ont.buildingsmart.no/idm_stages_alignment/" )
p = RDF.type
o = OWL.ontology
g4.add( ( s , p , o ) )


In [17]:
for i in range(3,stages_sheet.nrows):
    idm_stage = stages_sheet.cell_value(i,0)
    next_stage = stages_sheet.cell_value(i,6)
    riba_stage = stages_sheet.cell_value(i,9)

    j = i   
    while next_stage == "":
        j = j-1
        next_stage = stages_sheet.cell_value(j,6)
    print("idm stage {} maps to fasenorm stage {}".format(idm_stage,next_stage))
    steg = "Steg_"+str(next_stage).split(".")[0]
    idm_stage = "Stage_"+str(idm_stage).replace(".","_")
    
    #print( "Stage ID:" , "Steg"+stageID )
    s = idm_stages[ idm_stage ]
    p = RDF.type
    o = fasenorm [ steg ]
    g4.add( ( s , p , o ) )
    
    k = i
    while riba_stage == "":
        k=k-1
        riba_stage = stages_sheet.cell_value(k,9)
    if riba_stage not in ("","N/A"):
        r_stage = "Stage_"+str(riba_stage).split(".")[0]
        print("idm stage {} maps to Plan of Work stage {}".format(idm_stage,r_stage))

        p = RDF.type
        o = plan_of_work [ r_stage ]
        g4.add( ( s , p , o ) )
        

idm stage S00 maps to fasenorm stage 1.0
idm stage Stage_S00 maps to Plan of Work stage Stage_0
idm stage S01 maps to fasenorm stage 1.0
idm stage Stage_S01 maps to Plan of Work stage Stage_0
idm stage S02 maps to fasenorm stage 2.0
idm stage Stage_S02 maps to Plan of Work stage Stage_1
idm stage S03 maps to fasenorm stage 2.0
idm stage Stage_S03 maps to Plan of Work stage Stage_1
idm stage S04 maps to fasenorm stage 3.0
idm stage Stage_S04 maps to Plan of Work stage Stage_2
idm stage S05 maps to fasenorm stage 3.0
idm stage Stage_S05 maps to Plan of Work stage Stage_3
idm stage S06.1 maps to fasenorm stage 4.0
idm stage Stage_S06_1 maps to Plan of Work stage Stage_4
idm stage S06.2 maps to fasenorm stage 4.0
idm stage Stage_S06_2 maps to Plan of Work stage Stage_4
idm stage S07 maps to fasenorm stage 4.0
idm stage Stage_S07 maps to Plan of Work stage Stage_4
idm stage S08.1 maps to fasenorm stage 5.0
idm stage Stage_S08_1 maps to Plan of Work stage Stage_5
idm stage S08.2 maps to fase

In [18]:
# print file
g4.serialize( destination = "../filer/idm_stages_alignment.ttl" , format = "turtle" )
print( "Created idm_stages_alignment.ttl" )

Created idm_stages_alignment.ttl
